# Supersymmetric harmonic oscillator

As a warmup, lets go through the steps to get from the Hamiltonian of this system to a quantum circuit that lets us evolve states according to the time evolution operator.

The superpotential is 
\begin{equation}
    W(\hat{q}) = \frac{1}{2}m \hat{q}^2
\end{equation}
which leads to the following Hamiltonian,
\begin{equation}
    H = \frac{1}{2}\left(H_B+H_F\right), \quad 
        H_B=\hat{p}^2+m^2\hat{q}^2, \quad H_F=m\left[\hat{b}^{\dagger},\hat{b}\right].
\end{equation}
This contains two terms, a bosonic piece and a fermionic piece.  For the bosonic part, we could work in the position or momentum basis, but this is just the Hamiltonian for a harmonic oscillator, so lets convert to the number basis.  


In [1]:
import sys
sys.path.append('..')
from src.sympy_utilities import *

import sympy as sp

h_b = 0.5*(p*p + m*m*q*q)

print('Original H = ' + str(h_b))
h_harmonic = sp.expand(h_b.subs(qp_to_ada))
print('Using creation/annihlation operators H = ' + str(h_harmonic))

Original H = 0.5*(m**2*q**2 + p**2)
Using creation/annihlation operators H = 0.5*m*a*ad + 0.5*m*ad*a


And we know the matrix elements of $A$ and $A^{\dagger}$, they are defined in HamiltonianTerms.  We of course must impose a cutoff at this point, truncating the allowed excitations of the harmonic oscillator

In [2]:
h_matrix = convert(h_harmonic)

import numpy as np

matrix = np.array(h_matrix.subs({m: 1}).tolist())[:-1,:-1]
print(matrix)

[[0.500000000000000 0 0 0 0 0 0 0]
 [0 1.50000000000000 0 0 0 0 0 0]
 [0 0 2.50000000000000 0 0 0 0 0]
 [0 0 0 3.50000000000000 0 0 0 0]
 [0 0 0 0 4.50000000000000 0 0 0]
 [0 0 0 0 0 5.50000000000000 0 0]
 [0 0 0 0 0 0 6.50000000000000 0]
 [0 0 0 0 0 0 0 7.50000000000000]]


Now that we have the expression as a matrix, we can convert it to pauli strings, see \textbf{MatrixToPauliStrings} for the details.  

In [3]:
from src.MatrixToPauliString import *
from src.BinaryEncodings import *

pauli_strings = matrix_to_pauli_strings(matrix, standard_encode)
pauli_strings = sp.expand(sp.nsimplify(pauli_strings)) # numeric simplify
print(pauli_strings)

4*I^0*I^1*I^2 - 2*I^0*I^1*Z^2 - I^0*I^2*Z^1 - I^1*I^2*Z^0/2


But we can't forget about the fermionic piece!  We will always add the fermion onto the qubit after the highest numbered qubit for the boson.  

In [4]:
h_fermionic = m*b*bdag - m*bdag*b
print(h_fermionic)

m*b*bd - m*bd*b


In [5]:
fq = max_sympy_exponent(pauli_strings) + 1
b_ps = 0.5*sp.Symbol('X^'+str(fq),commutative=False)+0.5*complex(0,1)*sp.Symbol('Y^'+str(fq),commutative=False)
bdag_ps = 0.5*sp.Symbol('X^'+str(fq),commutative=False)-0.5*complex(0,1)*sp.Symbol('Y^'+str(fq),commutative=False)
commutation_lhs = sp.Symbol('X^'+str(fq), commutative=False)*sp.Symbol('Y^'+str(fq),commutative=False)-sp.Symbol('Y^'+str(fq), commutative=False)*sp.Symbol('X^'+str(fq),commutative=False)
commutation_rhs = 2*complex(0,1)*sp.Symbol('Z^'+str(fq))

In [6]:
fermionic_strings = sp.simplify(h_fermionic.subs({b: b_ps, bdag: bdag_ps})).subs(commutation_lhs, commutation_rhs)
print(fermionic_strings)

1.0*Z^3*m


In [7]:
h_tot = sp.expand(pauli_strings) + sp.expand(fermionic_strings)
print(h_tot)

4*I^0*I^1*I^2 - 2*I^0*I^1*Z^2 - I^0*I^2*Z^1 - I^1*I^2*Z^0/2 + 1.0*Z^3*m


At this point lets substitute in any parameters of the system (mass, coupling, etc).
This makes the qubit padding substantially easier given the way pauli strings are specified.

In [8]:
new_h = identity_qubit_padded_H(h_tot.subs(m,1))

In [9]:
print(new_h)

4*I^0*I^1*I^2*I^3 + 1.0*I^0*I^1*I^2*Z^3 - 2*I^0*I^1*I^3*Z^2 - I^0*I^2*I^3*Z^1 - I^1*I^2*I^3*Z^0/2


Now lets use our qiskit utilities!

In [10]:
from src.qiskit_utilities import pauli_string_to_trotter_step, pauli_string_to_op

print(pauli_string_to_op(new_h))
print(pauli_string_to_trotter_step(new_h,1))

SummedOp([
  -1.0 * IZII,
  -2.0 * IIZI,
  4.0 * IIII,
  -0.5 * ZIII,
  IIIZ
])
global phase: -4
      ┌───────┐   ┌───────┐  
q_0: ─┤ RZ(1) ├───┤ RZ(1) ├──
      ├───────┴┐  ├───────┴┐ 
q_1: ─┤ RZ(-2) ├──┤ RZ(-2) ├─
      ├────────┤  ├────────┤ 
q_2: ─┤ RZ(-1) ├──┤ RZ(-1) ├─
     ┌┴────────┴┐┌┴────────┴┐
q_3: ┤ RZ(-0.5) ├┤ RZ(-0.5) ├
     └──────────┘└──────────┘


In [11]:
import pickle

pickle.dump( new_h, open("DATA/ho_hamiltonian.p", "wb") )

# Supersymmetric Anharmonic Oscillator

The superpotential is 
\begin{equation}
    W(\hat{q}) = \frac{1}{2}m\hat{q}^2 + \frac{1}{4}g\hat{q}^4
\end{equation}

The full Hamiltonian is
\begin{equation}
    H=\frac{1}{2}\left[\hat{p}^2 + m^2\hat{q}^2 + 2mg\hat{q}^4 + g^2\hat{q}^6 - (m+3g\hat{q}^2)\left[b^{\dagger},b\right]\right]
\end{equation}

In [12]:
h_b = 0.5*(p*p + m*m*q*q + 2.*m*g*q*q*q*q + g*g*q*q*q*q*q*q)


print('Original H = ' + str(h_b))
h_harmonic = sp.expand(h_b.subs(qp_to_ada))
print('Using creation/annihlation operators H = ' + str(sp.simplify(h_harmonic)))

Original H = 0.5*(g**2*q**6 + 2.0*g*m*q**4 + m**2*q**2 + p**2)
Using creation/annihlation operators H = (0.0625*g**2*(a*ad*a*ad**2*a + a*ad*a*ad**3 + a*ad*a**2*ad*a + a*ad*a**2*ad**2 + a*ad*a**3*ad + a*ad*a**4 + a*ad**2*a*ad*a + a*ad**2*a**2*ad + a*ad**2*a**3 + a*ad**3*a*ad + a*ad**3*a**2 + a*ad**4*a + a*ad**5 + a**2*ad*a*ad**2 + a**2*ad*a**3 + a**2*ad**2*a*ad + a**2*ad**2*a**2 + a**2*ad**3*a + a**2*ad**4 + a**2*(ad*a)**2 + a**3*ad*a*ad + a**3*ad*a**2 + a**3*ad**2*a + a**3*ad**3 + a**4*ad*a + a**4*ad**2 + a**5*ad + a**6 + ad*a*ad*a**2*ad + ad*a*ad*a**3 + ad*a*ad**2*a*ad + ad*a*ad**2*a**2 + ad*a*ad**3*a + ad*a*ad**4 + ad*a**2*ad*a*ad + ad*a**2*ad**2*a + ad*a**2*ad**3 + ad*a**3*ad*a + ad*a**3*ad**2 + ad*a**4*ad + ad*a**5 + ad**2*a*ad*a**2 + ad**2*a*ad**3 + ad**2*a**2*ad*a + ad**2*a**2*ad**2 + ad**2*a**3*ad + ad**2*a**4 + ad**2*(a*ad)**2 + ad**3*a*ad*a + ad**3*a*ad**2 + ad**3*a**2*ad + ad**3*a**3 + ad**4*a*ad + ad**4*a**2 + ad**5*a + ad**6 + (a*ad)**2*a**2 + (a*ad)**3 + (a*ad**2)**2 + (a*

In [13]:
h_matrix = convert(sp.expand(h_harmonic))
matrix = np.array(h_matrix.subs({m: 1, g: 1}).tolist())[:-1,:-1]
print(matrix)

[[2.18750000000000 0 6.09879598773397 0 5.81753813911005 0
  1.67705098312484 0]
 [0 11.8125000000000 0 22.1985007939726 0 17.1163299220364 0
  4.43705983732471]
 [6.09879598773397 0 35.6875000000000 0 54.3430940874735 0
  36.7614777994574 0]
 [0 22.1985007939726 0 81.3125000000000 0 108.169788411552 0
  67.0226174511262]
 [5.81753813911005 0 54.3430940874735 0 156.187500000000 0
  189.306608937723 0]
 [0 17.1163299220364 0 108.169788411552 0 267.812500000000 0
  274.216340801383]
 [1.67705098312484 0 36.7614777994574 0 189.306608937723 0
  392.187500000000 0]
 [0 4.43705983732471 0 67.0226174511262 0 274.216340801383 0
  392.812500000000]]


In [14]:
pauli_strings = matrix_to_pauli_strings(matrix, standard_encode)
pauli_strings = sp.expand(sp.nsimplify(pauli_strings)) # numeric simplify removes very small numbers
print(pauli_strings)

335*I^0*I^1*I^2/2 + 2534359266234601*I^0*I^1*X^2/80000000000000 - 539*I^0*I^1*Z^2/4 + 49182024652081257*I^0*I^2*X^1/400000000000000 - 58*I^0*I^2*Z^1 + 3372539866389501*I^0*X^1*X^2/80000000000000 - 43522565295739943*I^0*X^1*Z^2/400000000000000 - 1617004543788743*I^0*X^2*Z^1/80000000000000 + 5707975433571519*I^0*Z^1*Z^2/80000000000000 - 335*I^1*I^2*Z^0/16 - 831198628691903*I^1*X^2*Z^0/80000000000000 + 57*I^1*Z^0*Z^2/8 - 10100943666989863*I^2*X^1*Z^0/400000000000000 - 75*I^2*Z^0*Z^1/8 - 5658670317827837*X^1*X^2*Z^0/400000000000000 + 6881002705742137*X^1*Z^0*Z^2/400000000000000 + 379246957374849*X^2*Z^0*Z^1/80000000000000 + 2243331453012137*Z^0*Z^1*Z^2/400000000000000


In [15]:
h_fermionic = -(m+3*g*q*q)*(b*bdag - bdag*b)
h_fermionic = h_fermionic.subs(qp_to_ada)
print(h_fermionic)

(-1.5*g*(a + ad)**2/m - m)*(b*bd - bd*b)


In [16]:
fq = max_sympy_exponent(pauli_strings) + 1
b_ps = 0.5*sp.Symbol('X^'+str(fq),commutative=False)+0.5*complex(0,1)*sp.Symbol('Y^'+str(fq),commutative=False)
bdag_ps = 0.5*sp.Symbol('X^'+str(fq),commutative=False)-0.5*complex(0,1)*sp.Symbol('Y^'+str(fq),commutative=False)
commutation_lhs = sp.Symbol('X^'+str(fq), commutative=False)*sp.Symbol('Y^'+str(fq),commutative=False)-sp.Symbol('Y^'+str(fq), commutative=False)*sp.Symbol('X^'+str(fq),commutative=False)
commutation_rhs = 2*complex(0,1)*sp.Symbol('Z^'+str(fq))

In [17]:
fermionic_strings = h_fermionic.subs({b*bdag - bdag*b: commutation_rhs})
print(fermionic_strings)

2.0*I*Z^3*(-1.5*g*(a + ad)**2/m - m)


In [18]:
sp.expand(fermionic_strings)

-3.0*I*Z^3*g*a*ad/m - 3.0*I*Z^3*g*a**2/m - 3.0*I*Z^3*g*ad*a/m - 3.0*I*Z^3*g*ad**2/m - 2.0*I*Z^3*m

In [19]:
h_matrix = convert(sp.expand(sp.simplify(fermionic_strings + 2.0*complex(0,1)*sp.Symbol('Z^'+str(fq))*m)))

matrix = np.array(h_matrix.subs({m: 1, g: 1}).tolist())[:-1,:-1]

print(matrix)

[[-3.0*I*Z^3 0 -4.24264068711929*I*Z^3 0 0 0 0 0]
 [0 -9.0*I*Z^3 0 -7.34846922834953*I*Z^3 0 0 0 0]
 [-4.24264068711929*I*Z^3 0 -15.0*I*Z^3 0 -10.3923048454133*I*Z^3 0 0 0]
 [0 -7.34846922834954*I*Z^3 0 -21.0*I*Z^3 0 -13.4164078649987*I*Z^3 0 0]
 [0 0 -10.3923048454133*I*Z^3 0 -27.0*I*Z^3 0 -16.431676725155*I*Z^3 0]
 [0 0 0 -13.4164078649987*I*Z^3 0 -33.0*I*Z^3 0 -19.4422220952236*I*Z^3]
 [0 0 0 0 -16.431676725155*I*Z^3 0 -39.0*I*Z^3 0]
 [0 0 0 0 0 -19.4422220952236*I*Z^3 0 -45.0*I*Z^3]]


In [20]:
f_pauli_strings = matrix_to_pauli_strings(matrix, standard_encode)
f_pauli_strings = sp.expand(sp.nsimplify(f_pauli_strings)) # numeric simplify
print(sp.N(f_pauli_strings))

-24.0*I*I^0*I^1*I^2*Z^3 + 12.0*I*I^0*I^1*Z^2*Z^3 - 11.8662521839619*I*I^0*I^2*X^1*Z^3 - 1.25e-15*I^0*I^2*Z^1*Z^3 + 6.0*I*I^0*I^2*Z^1*Z^3 - 5.952178177603*I*I^0*X^1*X^2*Z^3 + 6.07069722622744*I*I^0*X^1*Z^2*Z^3 - 1.25e-15*I^0*Z^1*Z^2*Z^3 - 5.952178177603*I*I^0*Z^1*Z^2*Z^3 + 3.0*I*I^1*I^2*Z^0*Z^3 + 1.52909347782471*I*I^2*X^1*Z^0*Z^3 + 1.25e-15*I^2*Z^0*Z^1*Z^3 + 0.75602575489635*I*X^1*X^2*Z^0*Z^3 + 0.0238207927904113*I*X^1*Z^0*Z^2*Z^3 + 1.25e-15*Z^0*Z^1*Z^2*Z^3 + 0.75602575489635*I*Z^0*Z^1*Z^2*Z^3


In [21]:
total_h = pauli_strings + f_pauli_strings - 2.0*complex(0,1)*sp.Symbol('Z^'+str(fq))

In [22]:
sp.N(total_h)

-24.0*I*I^0*I^1*I^2*Z^3 + 167.5*I^0*I^1*I^2 + 31.6794908279325*I^0*I^1*X^2 + 12.0*I*I^0*I^1*Z^2*Z^3 - 134.75*I^0*I^1*Z^2 - 11.8662521839619*I*I^0*I^2*X^1*Z^3 + 122.955061630203*I^0*I^2*X^1 - 1.25e-15*I^0*I^2*Z^1*Z^3 + 6.0*I*I^0*I^2*Z^1*Z^3 - 58.0*I^0*I^2*Z^1 - 5.952178177603*I*I^0*X^1*X^2*Z^3 + 42.1567483298688*I^0*X^1*X^2 + 6.07069722622744*I*I^0*X^1*Z^2*Z^3 - 108.80641323935*I^0*X^1*Z^2 - 20.2125567973593*I^0*X^2*Z^1 - 1.25e-15*I^0*Z^1*Z^2*Z^3 - 5.952178177603*I*I^0*Z^1*Z^2*Z^3 + 71.349692919644*I^0*Z^1*Z^2 + 3.0*I*I^1*I^2*Z^0*Z^3 - 20.9375*I^1*I^2*Z^0 - 10.3899828586488*I^1*X^2*Z^0 + 7.125*I^1*Z^0*Z^2 + 1.52909347782471*I*I^2*X^1*Z^0*Z^3 - 25.2523591674747*I^2*X^1*Z^0 + 1.25e-15*I^2*Z^0*Z^1*Z^3 - 9.375*I^2*Z^0*Z^1 + 0.75602575489635*I*X^1*X^2*Z^0*Z^3 - 14.1466757945696*X^1*X^2*Z^0 + 0.0238207927904113*I*X^1*Z^0*Z^2*Z^3 + 17.2025067643553*X^1*Z^0*Z^2 + 4.74058696718561*X^2*Z^0*Z^1 + 1.25e-15*Z^0*Z^1*Z^2*Z^3 + 0.75602575489635*I*Z^0*Z^1*Z^2*Z^3 + 5.60832863253034*Z^0*Z^1*Z^2 - 2.0*I*Z

In [23]:
new_h = identity_qubit_padded_H(total_h)

In [24]:
from src.qiskit_utilities import pauli_string_to_trotter_step, pauli_string_to_op

print(pauli_string_to_op(new_h))
print(pauli_string_to_trotter_step(new_h,1))

SummedOp([
  -58.0 * IZII,
  -108.80641323934985 * IXZI,
  -25.25235916747466 * ZXII,
  -14.146675794569592 * ZXXI,
  -20.212556797359287 * IZXI,
  -10.389982858648787 * ZIXI,
  -134.75 * IIZI,
  -20.9375 * ZIII,
  -9.375 * ZZII,
  -1.25e-15 * IZIZ,
  -1.25e-15 * IZZZ,
  1.25e-15 * ZZIZ,
  1.25e-15 * ZZZZ,
  7.125 * ZIZI,
  167.5 * IIII,
  4.740586967185612 * ZZXI,
  5.608328632530342 * ZZZI,
  31.67949082793251 * IIXI,
  42.15674832986876 * IXXI,
  71.34969291964399 * IZZI,
  17.20250676435534 * ZXZI,
  122.95506163020315 * IXII,
  0.0 * ZIIZ,
  0.0 * IIZZ,
  0.0 * IXIZ,
  0.0 * IXXZ,
  0.0 * ZXXZ,
  0.0 * ZXZZ,
  0.0 * IXZZ,
  0.0 * ZXIZ,
  0.0 * IIIZ
])
global phase: -4.1372
     ┌───────┐┌───┐┌───┐┌───────┐┌───┐┌───┐┌───┐     ┌───┐┌───────┐┌───┐┌───┐»
q_0: ┤ RZ(0) ├┤ X ├┤ X ├┤ RZ(0) ├┤ X ├┤ X ├┤ X ├─────┤ X ├┤ RZ(0) ├┤ X ├┤ X ├»
     └───────┘└─┬─┘└─┬─┘└───────┘└─┬─┘└─┬─┘└─┬─┘     └─┬─┘└───────┘└─┬─┘└─┬─┘»
q_1: ───────────┼────┼─────────────┼────┼────■─────────┼─────────────┼────■─

In [25]:
import pickle

pickle.dump( new_h, open("DATA/aho_hamiltonian.p", "wb") )